In [2]:
def draw_multiplex_network_communities(layers,k_):
    
    layer12_concat = pd.concat(layers,axis=0)
    nodes, edges = network_reformat(layer12_concat, weighted=True, output_format='graph',node_format="hash")
    
    hash_to_int = dict(zip(nodes,range(len(nodes))))
    int_to_hash = dict(zip(range(len(nodes)),nodes))
    
    G = nx.Graph()
    G.add_nodes_from([hash_to_int[n] for n in nodes])
    G.add_weighted_edges_from([(hash_to_int[i],hash_to_int[j], w) for i,j,w in edges])
    #nx.set_node_attributes(G, 'community', node_communities)
    
    
    node_communities = [set([hash_to_int[n] for n in c]) for c in community_detection_multiplex(*layers)]
    
    
    return node_communities
    
    
    
    return nx.get_node_attributes(G, 'community')
    
    # Initialise high-level graph
    G_communities = nx.Graph()

    community_nodes = list(set(communities))
    
    # Find edges in the community network (slightly more complicated)
    community_edges = {} #Hash table because it makes collapsing edges easier
    internal_edges = [0] * num_communities #Counting num of links within each cluster

    for (i,j) in G.edges():
        
        c_i = communities[i]
        c_j = communities[j]
        
        w_ij = G.get_edge_data(i,j)['weight']
        
        if c_i != c_j:
            key = ' '.join([str(num) for num in sorted([c_i,c_j])]) # Example: '1 2'
            if key not in community_edges:
                community_edges[key] = w_ij
            else:
                community_edges[key] += w_ij
        else:
            internal_edges[c_i] += w_ij
    
    community_edges = [tuple([int(e) for e in k.split(" ")]+[v]) for (k,v) in community_edges.items()] # Horrible I know..
    
    G_communities.add_nodes_from(community_nodes)
    G_communities.add_weighted_edges_from(community_edges)
    
    
    ###############
    # BOILERPLATE #
    ###############
    
    pos = nx.spring_layout(G_communities, k=k_, iterations=10)
    
    # color map from http://colorbrewer2.org/
    cmapLight = colors.ListedColormap(['#a6cee3', '#b2df8a', '#fb9a99', '#fdbf6f', '#cab2d6'], 'indexed', num_communities)
    cmapDark = colors.ListedColormap(['#1f78b4', '#33a02c', '#e31a1c', '#ff7f00', '#6a3d9a'], 'indexed', num_communities)

    # edges
    weights_transformed = np.log(np.array([w for (i,j,w) in community_edges]))**3+100
    nx.draw_networkx_edges(G_communities, 
                           pos,
                           width=weights_transformed/max(weights_transformed)
    )

    # nodes
    nodeCollection = nx.draw_networkx_nodes(G_communities,
        pos = pos,
        node_color = G_communities.nodes(),
        cmap = cmapLight,
        node_size = ((np.array(internal_edges))*1.85/(np.array(internal_edges)).max()+0.15)*1000
    )
    # set node border color to the darker shade
    darkColors = [cmapDark(v) for v in G_communities.nodes()]
    nodeCollection.set_edgecolor(darkColors)
    
    # Node labels
    for n in G_communities.nodes_iter():
        plt.annotate(n,
            xy = pos[n],
            textcoords = 'offset points',
            horizontalalignment = 'center',
            verticalalignment = 'center',
            xytext = [0, 2],
            color = cmapDark(n)
        )

    plt.axis('off')
    #plt.savefig("netw.png")
    
    
#draw_multiplex_network_communities([layer1,layer2],k_=2)

In [29]:
def network_reformat_multiplex(halflife=-1,node_representation="physical",*args):
    """Return multiplex representation of multiplex network
    
    Parameters
    ----------
    halflife : number
        Halflife in seconds of relax-rate decay between layers.
        Defaults to -1.
    node_representation : string
        Set to "state if state nodes associated with the same physical
        node should have different reference names. Defaults to "physical".
    *args : pandas df formatted layers
        
    Returns
    -------
    net_file : string
        A network string in multiplex format
    int_to_hash : dict
        Key-value pairs of node integer id and original hash id
    """
    print "-> network_reformat_multiplex\n\t",
    
    # Infomap will only work with node indices on the command line.
    
    # Pool all nodes and sort the set
    nodes_hash = []
    for l, df in enumerate(args):
        layer_nodes = set()
        layer_nodes.update(df["user1"])
        layer_nodes.update(df["user2"])

        if node_representation == "state":
            # Add layer identifier to end of each node hash
            layer_nodes = [hash_str+"_"+str(l+1) for hash_str in layer_nodes]    
        nodes_hash.extend(layer_nodes) # no good reason to sort them
        
    if node_representation == "physical":
        nodes_hash = sorted(set(nodes_hash)) # no good reason not to sort them
    
    nodes = nodes_hash
    
    # Add vertices
    out_file = "*Vertices %d" % len(nodes)
    
    hashid_to_intid = {}
    for i,n in enumerate(nodes):
        intid = i+1
        hashid = str(n)
        out_file += '\n%d "Node %s" 1.0' % (intid,hashid)
        hashid_to_intid[hashid] = intid
        
    # Add edges header to file
    out_file += "\n*Multiplex"
    
    for l, df in enumerate(args):
        if node_representation == "state":
            user1 = [hash_str+"_"+str(l+1) for hash_str in df["user1"]]
            user2 = [hash_str+"_"+str(l+1) for hash_str in df["user2"]]
        else:
            user1 = df["user1"]
            user2 = df["user2"]
            
        edges_hash = zip(user1, user2)
        
        # Add weights. THIS IS REDUNDANT FOR 5MINS TIMESLICES BECAUSE PPL ONLY MEET ONCE HERE.
        edges = [(e[0],e[1],w) for e,w in Counter(edges_hash).items()]
        
        # Add edges to file
        for i,j,w in edges:
            out_file += '\n%d %s %d %s %d' % (l+1,hashid_to_intid[i],l+1,hashid_to_intid[j],w) #+1 because 1 is first layer index
            
    # Add interlayer relax-rates - if halflife is not infinte (represented as -1)
    if halflife == -1:
        return out_file
    
    # Relax decay function
    def N(t):
        tau = halflife/np.log(2)
        return np.exp(-t/float(tau))
    
    # Add inter-statenode relax rates
    if node_representation == "state":
        nodes_hash_set = set([hash_str_id.split("_")[0] for hash_str_id in nodes_hash])
        error_count = 0
        for n in nodes_hash_set:
            for l1, df1 in enumerate(args):
                for l2, df2 in enumerate(args):
                    if not l2 > l1:
                        continue
                        
                    in_both_layers = set(list(df1['user1'].values)+list(df1['user2'].values)) & set(list(df2['user1'].values)+list(df2['user2'].values))
                    if n not in in_both_layers:
                        continue
                        
                    n1 = n+"_"+str(l1+1)
                    n2 = n+"_"+str(l2+1)
                    
                    time_diff = df2['timestamp'].values[0] - df1['timestamp'].values[0]
                    out_file += '\n%d %s %d %s %f' % (l1+1,hashid_to_intid[n1],l2+1,hashid_to_intid[n2],N(time_diff))
                    
                        
    elif node_representation == "physical":
        for n in nodes:
            for l1, df1 in enumerate(args):
                for l2, df2 in enumerate(args):
                    if not l2 > l1:
                        continue
                    time_diff = df2['timestamp'].values[0] - df1['timestamp'].values[0]
                    out_file += '\n%d %s %d %s %f' % (l1+1,hashid_to_intid[n],l2+1,hashid_to_intid[n],N(time_diff))
    
    
    return out_file

In [2]:
from graphtool import *

ImportError: No module named graph_tool